In [1]:

import pandas as pd
from muraves_lib import run_manager

ModuleNotFoundError: No module named 'muraves_lib'

In [6]:


df = run_manager.create_database(raw_gz_path='/group/Muography/abiolchi/test/RAW_GZ')

print(df.head())
print(list(df["timestamp"]))



100%|██████████| 11/11 [00:00<00:00, 1742.94it/s]

  hodoscope    run      timestamp         day   hour    wp  temperature  \
0       BLU  10089  1663863806075  2022-09-22  18:23  25.0         22.7   
1       BLU  10081  1663826992630  2022-09-22  08:09  20.0         17.6   
2       BLU  10080  1663822958694  2022-09-22  07:02  20.0         17.8   
3       BLU  10087  1663853236993  2022-09-22  15:27  25.0         23.2   
4       BLU  10084  1663839121954  2022-09-22  11:32  20.0         20.3   

   humidity  trigger_rate  accidental_rate orientation  
0      51.4         8.617            0.000     fre_sky  
1      70.0        12.933            0.250     fre_sky  
2      68.7        12.383            0.167     fre_sky  
3      51.3         8.233            0.000     fre_sky  
4      58.4        12.483            0.250     fre_sky  
[1663863806075, 1663826992630, 1663822958694, 1663853236993, 1663839121954, 1663858553653, 1663835067114, 1663842394942, 1663847863467, 1663831053489, 1572143168320, 1630138733053, 1572121917353, 16563849271

In [ ]:
# inquiry

def select_by_date(df, start_day, end_day) -> pd.DataFrame:
    # Convert input to datetime.date
    start_day = pd.to_datetime(start_day).date()
    end_day   = pd.to_datetime(end_day).date()

    df = df.copy()
    df["day"] = pd.to_datetime(df["day"]).dt.date  # force conversion always

    print("Start:", start_day, "End:", end_day)


    mask = (df["day"] >= start_day) & (df["day"] <= end_day)
    print("Rows selected:", mask.sum(), "out of", len(df))

    return df[(df["day"] >= start_day) & (df["day"] <= end_day)]


result = select_by_date(df, "2020-10-20", "2022-10-30")
print(result)

df.isna().any() 
df.isna().sum()        # count per column





In [1]:
# EXAMPLE: Save ROOT with metadata
import ROOT
from utils import root_wrapper

# creo un istogramma dummy
h = ROOT.TH1F("h", "Example", 100, 0, 1)
h.FillRandom("gaus", 1000)

# genero sample con metadati
meta = root_wrapper.save_root_with_metadata("sample.root", h)
print("Metadati salvati:", meta)


fatal: No names found, cannot describe anything.
fatal: No names found, cannot describe anything.


Metadati salvati: {'algorithms': {'v0.0.': None}, 'generated_at': '2025-12-12T16:46:05.388082', 'commit': '14dfbd4'}


In [2]:
root_wrapper.load_root_metadata("sample.root")

{'algorithms': {'v0.0.': None},
 'generated_at': '2025-12-12T16:46:05.388082',
 'commit': '14dfbd4'}